# <center> Innoscape - Telecom Paristech - Projet fil rouge </center>
# <center> Mapping Groupes de produits </center>

In [61]:
# Librairies 
import pandas as pd
import numpy as np
from scipy import stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import FrenchStemmer
import re
import random
from Preprocessing2 import *
import gensim.models as models
from gensim import corpora
from gensim import matutils

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 60)

#chemin = "../Data/"
chemin = "C:/Users/JN/Desktop/Telecom ParisTech/Cours/fil_rouge/github/Data/"
chemin_val = "/Users/valentinphetchanpheng/Documents/MS BGD Telecom ParisTech/Projet Fil Rouge/"

## Importation des données et reformatage

In [2]:
# Importation des données vendeurs
df_seller_origin = pd.read_csv(chemin + "t3_all_fr_prices_posweekly_prod_without_price.csv", sep = ",")

#Création d'un df allégé pour le travail - on ne retient que les colonnes identifiées comme utiles pour l'analyse à ce stade
columns_to_keep = ['t3_all_fr_prices_posweekly_prod.productfamily_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory4_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory3_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory2_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory1_seller',
                   't3_all_fr_prices_posweekly_prod.productseller_name',
                   't3_all_fr_prices_posweekly_prod.id_source',
                   't3_all_fr_prices_posweekly_prod.brand_seller']

df_seller = df_seller_origin[columns_to_keep]

#on raccourcit un peu les noms des colonnes en ôtant les préfixes redondants
shortened_names = map(lambda string: string.replace('t3_all_fr_prices_posweekly_prod.',''), columns_to_keep)
col_name_dict = dict(zip(columns_to_keep, shortened_names))
df_seller.rename(columns=col_name_dict, inplace=True)

In [3]:
# Importation des données références catégories Innoscape
df_ref_origin = pd.read_csv(chemin + "2018.11.21_innoscape_internal_architecture.csv")
df_ref = df_ref_origin.drop("Date", axis=1)

#On garde que la colonne correspondante à la famille de produit de référence finale
columns_to_keep = ['innoscape_ref_productfamily_seller']
df_ref = df_ref[columns_to_keep]
df_ref = df_ref.drop_duplicates()

In [4]:
df_seller = df_seller.dropna(how='all', subset=['productfamily_seller',
                                                'prodcategory4_seller',
                                                'prodcategory3_seller',
                                                'prodcategory2_seller',
                                                'prodcategory1_seller',
                                                'productseller_name'])
df_seller = df_seller.fillna("")

# Retrait des caractères spéciaux et de la ponctuation
spec_char = {'à':'a','â':'a','ä':'a',
              'é':'e','è':'e','ê':'e','ë':'e',
              'ï':'i','î':'i',
              'ô':'o','ö':'o',
              'ù':'u','û':'u',
              'ç':'c',
              ',':' ', "'":" ", ";":" ", ":":" ", "\?":" ", "!":" ", "\.":'', "  ":" ", 
              "\(":"","\)":"","\[":"","\]":"","-":" ","\+":"","/":"","\*":"","–":" ",
              "&":"et"
              }


In [5]:
# Importation de la liste de stop words
f = open("stop_words.txt", "r")
stop_words = set(f.read().split('\n'))

g = open("marques.txt", "r")
marques = set(g.read().split('\n'))

stop_words = stop_words | marques

In [8]:
#preprocessing des données
df_ref=preprocessing(df_ref,df_ref.columns, spec_char, stop_words)
list_new_col = ['productfamily_seller', 'prodcategory4_seller', 'prodcategory3_seller', 'prodcategory2_seller', 'prodcategory1_seller', 'productseller_name']
df_seller = preprocessing(df_seller, list_new_col, spec_char, stop_words)

In [9]:
df_seller.head(10)

productfamily_seller   prodcategory4_seller  \
0     Colles à carrelage     Colles à carrelage   
1                                                 
2                                                 
3                 Mastic                 Mastic   
4                  Cires                  Cires   
5                                                 
6      Perceuse-visseuse              Perceuses   
7  Produits d'étanchéité  Produits d'étanchéité   
8              Arroseurs              Arroseurs   
9                                                 

               prodcategory3_seller                      prodcategory2_seller  \
0                Colles et adhésifs                             Quincaillerie   
1                                                                               
2  Accessoires pour lave-vaisselles                               Accessoires   
3                Colles et adhésifs                             Quincaillerie   
4  Peintures, teintures et solvants  Peintures, outils et traitement des murs   
5                                    Peintures, outils et traitement des murs   
6         Outillage électroportatif       Outillage à main et électroportatif   
7                Colles et adhésifs                             Quincaillerie   
8               Matériel d'arrosage                                 Jardinage   
9                                                                               

  prodcategory1_seller                                 productseller_name  \
0            Bricolage  SikaCeram Xtra - Colle à carrelage intérieur e...   
1                       Ariston 3200362 scaldabagno elettrico PRO 100 H/3   
2  Gros électroménager  AEG Panier À Couverts Lave-vaisselle Cage & Ha...   
3            Bricolage              Colle-mastic pour joints d'étanchéité   
4            Bricolage  Sikagard Protection Sol MAT - Imperméabilisant...   
5            Bricolage  Sikagard Protection Toiture inclinée - Protect...   
6            Bricolage  Perceuse-visseuse RYOBI 18V OnePlus - 1 batter...   
7            Bricolage  Sika Multiseal - Bande d'étanchéité autocollan...   
8               Jardin            Black and Decker 34345 Arroseur Impulse   
9                       Ryobi-RXB01 Cortasetos-Écartement des dents La...   

  id_source      brand_seller productfamily_seller_formatted  \
0    AMZ_FR     SIKAFRANCESAS             colles a carrelage   
1    AMZ_FR  ARISTONTHERMOSPA                                  
2    AMZ_FR               AEG                                  
3    AMZ_FR     SIKAFRANCESAS                         mastic   
4    AMZ_FR     SIKAFRANCESAS                          cires   
5    AMZ_FR     SIKAFRANCESAS                                  
6    AMZ_FR             RYOBI              perceuse visseuse   
7    AMZ_FR     SIKAFRANCESAS          produits d etancheite   
8    AMZ_FR       BLACKDECKER                      arroseurs   
9    AMZ_FR             RYOBI                                  

  prodcategory4_seller_formatted    prodcategory3_seller_formatted  \
0             colles a carrelage                colles et adhesifs   
1                                                                    
2                                 accessoires pour lave vaisselles   
3                         mastic                colles et adhesifs   
4                          cires   peintures teintures et solvants   
5                                                                    
6                      perceuses         outillage electroportatif   
7          produits d etancheite                colles et adhesifs   
8                      arroseurs               materiel d arrosage   
9                                                                    

            prodcategory2_seller_formatted prodcategory1_seller_formatted  \
0                            quincaillerie                      bricolage   
1                                                                       

In [10]:
cols_to_keep = ['productfamily_seller',
                'prodcategory4_seller',
                'prodcategory3_seller',
                'prodcategory2_seller',
                'prodcategory1_seller',
                'productseller_name',
                'id_source',
                'productfamily_seller_stem',
                'prodcategory4_seller_stem',
                'prodcategory3_seller_stem',
                'prodcategory2_seller_stem',
                'prodcategory1_seller_stem',
                'productseller_name_stem']

df2 = df_seller[cols_to_keep]
df2.rename(columns={'productfamily_seller_stem':"prod_fam",
                'prodcategory4_seller_stem':'cat4',
                'prodcategory3_seller_stem':'cat3',
                'prodcategory2_seller_stem':'cat2',
                'prodcategory1_seller_stem':'cat1',
                'productseller_name_stem':'description'}, inplace=True)

In [11]:
df2.head()

productfamily_seller prodcategory4_seller              prodcategory3_seller  \
0   Colles à carrelage   Colles à carrelage                Colles et adhésifs   
1                                                                               
2                                            Accessoires pour lave-vaisselles   
3               Mastic               Mastic                Colles et adhésifs   
4                Cires                Cires  Peintures, teintures et solvants   

                       prodcategory2_seller prodcategory1_seller  \
0                             Quincaillerie            Bricolage   
1                                                                  
2                               Accessoires  Gros électroménager   
3                             Quincaillerie            Bricolage   
4  Peintures, outils et traitement des murs            Bricolage   

                                  productseller_name id_source  \
0  SikaCeram Xtra - Colle à carrelage intérieur e...    AMZ_FR   
1  Ariston 3200362 scaldabagno elettrico PRO 100 H/3    AMZ_FR   
2  AEG Panier À Couverts Lave-vaisselle Cage & Ha...    AMZ_FR   
3              Colle-mastic pour joints d'étanchéité    AMZ_FR   
4  Sikagard Protection Sol MAT - Imperméabilisant...    AMZ_FR   

           prod_fam              cat4                       cat3  \
0  [coll, carrelag]  [coll, carrelag]              [coll, adhes]   
1                []                []                         []   
2                []                []  [accessoir, lav, vaissel]   
3          [mastic]          [mastic]              [coll, adhes]   
4             [cir]             [cir]   [teintur, solv, peintur]   

                           cat2                  cat1  \
0                 [quincailler]            [bricolag]   
1                            []                    []   
2                   [accessoir]  [gros, electromenag]   
3                 [quincailler]            [bricolag]   
4  [mur, trait, peintur, outil]            [bricolag]   

                                         description  
0  [carrelag, max, carreau, coll, sikaceram, inte...  
1                      [scaldabagno, pro, elettrico]  
2  [couvert, handl, vaissel, couvercl, cag, lav, ...  
3                   [etancheit, joint, coll, mastic]  
4          [effet, protect, impermeabilis, mat, sol]

In [12]:
#concaténation de tous les champs pertinents en un seul texte par produit
# df2['prod_info'] = df2.prod_fam + df2.cat4 +df2.description
# df2['prod_info'] = df2['prod_info'].apply(lambda x: ' '.join(x))
# df2['prod_info']

df2['prod_info'] = df2.description

## TF-IDF

In [13]:
#tfidf version gensim
texts = [document for document in df2['prod_info']]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)
vectors = tfidf[corpus]

## Embeddings

In [52]:
# embedding lda
num_topics = 10
lda = models.LdaModel(corpus = vectors,
                      id2word=dictionary,
                      num_topics=num_topics,
                      chunksize = 3000,
                      alpha='auto',
                      random_state=0
                      )
corpus_lda = lda[vectors]

In [53]:
lda.show_topics(100)

[(0,
  '0.034*"tuil" + 0.030*"roman" + 0.029*"section" + 0.028*"lantern" + 0.026*"chatier" + 0.026*"cm²" + 0.023*"ventil" + 0.022*"pureau" + 0.017*"model" + 0.017*"roug"'),
 (1,
  '0.049*"areti" + 0.044*"faitier" + 0.041*"about" + 0.036*"terr" + 0.033*"cuit" + 0.031*"rond" + 0.027*"angulair" + 0.026*"emboit" + 0.023*"dem" + 0.020*"roug"'),
 (2,
  '0.069*"cuit" + 0.064*"terr" + 0.032*"tuil" + 0.029*"roug" + 0.026*"rustiqu" + 0.022*"individuel" + 0.021*"flamm" + 0.020*"brun" + 0.017*"douill" + 0.017*"seau"'),
 (3,
  '0.034*"mass" + 0.031*"perfor" + 0.029*"sd" + 0.023*"sci" + 0.022*"pann" + 0.021*"omeg" + 0.018*"coniqu" + 0.017*"huguenot" + 0.014*"circulair" + 0.014*"raccord"'),
 (4,
  '0.054*"cuit" + 0.050*"terr" + 0.050*"riv" + 0.037*"rabat" + 0.035*"droit" + 0.034*"tuil" + 0.032*"gauch" + 0.023*"vieil" + 0.020*"roug" + 0.016*"bourgogn"'),
 (5,
  '0.033*"silvacan" + 0.029*"littoral" + 0.028*"fonc" + 0.022*"closoir" + 0.017*"cuit" + 0.016*"terr" + 0.015*"fibr" + 0.015*"diametr" + 0.014*"

## Test de l'embedding sur corpus annoté

In [184]:
#importation du corpus annoté
df_annote = pd.read_excel(chemin + "Train_set.xlsx")
cols_to_keep = ['Column1', 
                'productfamily_seller', 
                'productseller_name',
                'Product_category_1_innoscape', 
                'Product_category_2_innoscape',
                'Product_family_innoscape']
df_annote = df_annote[cols_to_keep]
df_annote.head()

Column1 productfamily_seller  \
0        0   Colles à carrelage   
1       28   Colles à carrelage   
2       49   Colles à carrelage   
3       63   Colles à carrelage   
4        3               Mastic   

                                  productseller_name  \
0  SikaCeram Xtra - Colle à carrelage intérieur e...   
1  SikaCeram Joint Fin - Mortier pour joint de ca...   
2  SikaCeram Joint Dallage extérieur - Mortier jo...   
3  Colle à carrelage en pâte spéciale carreaux et...   
4              Colle-mastic pour joints d'étanchéité   

  Product_category_1_innoscape Product_category_2_innoscape  \
0      Matériaux et Menuiserie   Colles-Adhésifs-Etanchéité   
1      Matériaux et Menuiserie   Colles-Adhésifs-Etanchéité   
2      Matériaux et Menuiserie   Colles-Adhésifs-Etanchéité   
3      Matériaux et Menuiserie   Colles-Adhésifs-Etanchéité   
4      Matériaux et Menuiserie   Colles-Adhésifs-Etanchéité   

    Product_family_innoscape  
0  Mastic, colle, étanchéité  
1  Mastic, colle, étanchéité  
2  Mastic, colle, étanchéité  
3  Mastic, colle, étanchéité  
4  Mastic, colle, étanchéité

In [179]:
dictionary

In [185]:
#on effectue le même préprocessing sur les données annotées que précédemment:
list_new_col2 = ['productfamily_seller', 'productseller_name','Product_family_innoscape']
df_test = preprocessing(df_annote, list_new_col2, spec_char, stop_words)
df_test['prod_info'] = df_test['productfamily_seller_stem'] + df_test['productseller_name_stem']
df_test = df_test[['Column1','productfamily_seller','productseller_name', 'prod_info','Product_family_innoscape']]
df_test.set_index('Column1',inplace=True)
df_test.head()

productfamily_seller  \
Column1                        
0         Colles à carrelage   
28        Colles à carrelage   
49        Colles à carrelage   
63        Colles à carrelage   
3                     Mastic   

                                        productseller_name  \
Column1                                                      
0        SikaCeram Xtra - Colle à carrelage intérieur e...   
28       SikaCeram Joint Fin - Mortier pour joint de ca...   
49       SikaCeram Joint Dallage extérieur - Mortier jo...   
63       Colle à carrelage en pâte spéciale carreaux et...   
3                    Colle-mastic pour joints d'étanchéité   

                                                 prod_info  \
Column1                                                      
0        [coll, carrelag, carrelag, max, carreau, coll,...   
28       [coll, carrelag, carrelag, fin, cendr, gach, s...   
49       [coll, carrelag, carrelag, gris, terr, sikacer...   
63       [coll, carrelag, pat, carrelag, special, carre...   
3                 [mastic, etancheit, joint, coll, mastic]   

          Product_family_innoscape  
Column1                             
0        Mastic, colle, étanchéité  
28       Mastic, colle, étanchéité  
49       Mastic, colle, étanchéité  
63       Mastic, colle, étanchéité  
3        Mastic, colle, étanchéité

In [186]:
#on exprime le champ prod_info de df_test dans le système de coordonnées de l'embedding:
texts_test = [document for document in df_test['prod_info']]
corpus_test = [dictionary.doc2bow(text) for text in texts_test]
tfidf_test = models.TfidfModel(corpus_test)
vectors_test = tfidf[corpus_test]
corpus_test_lda = lda[vectors_test]

In [157]:
#on intègre les coordonnées de l'embedding dans df_test:
coords = pd.DataFrame(matutils.corpus2csc(corpus_test_lda).T.todense(),index=df_test.index.values)
df_test = pd.concat([df_test, coords], axis=1, join='inner')
df_test.rename(columns=dict(zip(range(num_topics),['topic_'+str(i) for i in range(num_topics)])),inplace=True)
df_test.head()

#on calcule les barycentres de chaque famille Innoscape dans le système de coordonnées de l'embedding:
families = df_test['Product_family_innoscape'].unique()
fam_indices = dict(zip(families,
                       [[i for i in df_test.index 
                         if df_test.loc[i,'Product_family_innoscape']==f] 
                        for f in families]))

centroids = df_test.groupby('Product_family_innoscape').mean()

In [166]:
col_list = ['topic_'+str(i) for i in range(num_topics)]
df_test['distances'] = df_test.apply(sq_distances, 
                               args=('Product_family_innoscape',
                                     col_list,
                                     centroids),
                                     axis=1)

Column1
0        0.484807
28       0.127653
49       0.383890
63       0.101637
3        0.194165
57       0.105648
85       0.436612
108      0.298823
4        0.112342
186      0.085303
198      0.143294
408      0.236636
6        0.253137
79       0.234735
161      0.054554
203      0.291184
7        0.097005
41       0.047647
250      0.082792
255      0.097080
8        0.071290
145      0.019118
148      0.020674
183      0.031196
11       0.253023
12       0.165720
109      0.069490
263      0.092315
328      0.060802
13       0.150203
           ...   
20740    0.082251
19451    0.239454
20462    0.080541
22258    0.239503
24296    0.145922
19567    0.174863
19627    0.174826
22331    0.091677
22758    0.159908
20750    0.115426
22192    0.071161
22321    0.183052
30724    0.114290
30725    0.184013
30731    0.153953
30735    0.154907
30726    0.090606
30728    0.118960
30733    0.109622
30737    0.097613
30729    0.282361
30730    0.249163
30734    0.396134
30736    0.093645
30

In [170]:
#calcul des distances moyennes
avg_sq_dist = np.mean(df_test['distances'])
print('average square distance to the cluster centroid: '+str(avg_sq_dist))

avg_sq_dist_by_fam = df_test.groupby('Product_family_innoscape').mean()['distances']
avg_sq_dist_by_fam

average square distance to the cluster centroid: 0.14966270054589653


Product_family_innoscape
Abris de jardin                                                               0.163682
Accessoires amenagement et produits d'entretien                               0.153399
Accessoires chauffe-eau                                                       0.136135
Accessoires couverture                                                        0.202470
Accessoires d'isolation                                                       0.149434
Accessoires du tableau électrique (bornier, peigne,etc)                       0.099607
Accessoires et consommables outillage électroportatif                         0.158011
Accessoires et divers outils peintre                                          0.099635
Accessoires gros oeuvre                                                       0.062808
Accessoires outillage stationnaire                                            0.100920
Accessoires plein air                                                         0.000000
Accessoires pour e